In [1]:
# ! pip install matplotlib transformers datasets opencv-python h5py accelerate sckit-learn

### Importing the necessary python libraries

In [1]:
import numpy as np
import h5py
from datasets import Dataset, load_dataset
import pandas as pd
import cv2
import torch
import math
from matplotlib import pyplot as plt

c:\Users\ATI-G2\Envs\trans\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sequence import ECGSequence

### defining the paths

In [3]:
hdf5_path =  r"C:\Users\ATI-G2\Documents\python\ECG\data\code-15\exams_part0.hdf5"
exams_csv_path = r"C:\Users\ATI-G2\Documents\python\ECG\data\code-15\exams.csv"

In [4]:
f = h5py.File(hdf5_path,"r")
pd_results = pd.read_csv(exams_csv_path,index_col="exam_id")


### Rough

In [3]:
# path_to_csv = r"C:\Users\ATI-G2\Documents\python\ECG\data\code-15\exams.csv"
# pd_results = pd.read_csv(path_to_csv,index_col="exam_id")
# pd_results.index
# for idx in pd_results.index:
#     print(idx)
#     break

1169160


### Transformers and datasets using hugging faces

In [33]:

data_root_fldr = r"C:\Users\ATI-G2\Documents\python\ECG\Transformers\data"

food = load_dataset("imagefolder",
                    data_dir=data_root_fldr
                    )


Resolving data files: 100%|██████████| 20000/20000 [00:00<00:00, 23845.59it/s]


In [27]:
data = food.train_test_split(test_size=0.05)

In [34]:
food

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 20000
    })
})

In [15]:
# from sklearn.model_selection import train_test_split

# train_data, validation_data = train_test_split(
#     food['train'],
#     test_size=0.05,
#     random_state=42  # Set a random seed for reproducibility
# )

In [18]:
del train_data 
# del valdation_data

NameError: name 'train_data' is not defined

### Working on extracting the labels for the generated images into a csv file

In [24]:
pd_results.head(1)

,age,is_male,nn_predicted_age,1dAVb,RBBB,LBBB,SB,ST,AF,patient_id,death,timey,normal_ecg,trace_file
exam_id,,,,,,,,,,,,,,
1169160,38,True,40.160484,False,False,False,False,False,False,523632,False,2.098628,True,exams_part13.hdf5


Filtering the dataframe to extract only the required columns

In [134]:
# exam_id = np.delete(np.array(f["exam_id"]),np.where(np.array(f["exam_id"])==0)).tolist()
# pd_req = pd_results.loc[exam_id,["1dAVb","RBBB","LBBB","SB","ST","AF","normal_ecg"]]
# labels_arr = pd_req.to_numpy()


In [6]:
pd_req = pd_results[pd_results["trace_file"]=="exams_part0.hdf5"][["1dAVb","RBBB","LBBB","SB","ST","AF","normal_ecg"]]
labels_arr = pd_req.to_numpy()
exam_id = pd_req.index.to_list()



In [56]:
pd_req.info(), pd_req.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 590673 to 536312
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   1dAVb       20000 non-null  bool 
 1   RBBB        20000 non-null  bool 
 2   LBBB        20000 non-null  bool 
 3   SB          20000 non-null  bool 
 4   ST          20000 non-null  bool 
 5   AF          20000 non-null  bool 
 6   normal_ecg  20000 non-null  bool 
dtypes: bool(7)
memory usage: 293.0 KB


(None,
         1dAVb   RBBB   LBBB     SB     ST     AF normal_ecg
 count   20000  20000  20000  20000  20000  20000      20000
 unique      2      2      2      2      2      2          2
 top     False  False  False  False  False  False      False
 freq    19655  19462  19663  19682  19577  19554      12299)

In [7]:
df = pd_req

# Create a summary list of dictionaries
summary = []

for column in df.columns:
    unique_counts = df[column].value_counts().to_dict()
    summary.append({'Column': column, 'Unique_Values': unique_counts})

# Create a summary DataFrame from the list of dictionaries
summary_df = pd.DataFrame(summary)

print(summary_df)


       Column               Unique_Values
0       1dAVb   {False: 19655, True: 345}
1        RBBB   {False: 19462, True: 538}
2        LBBB   {False: 19663, True: 337}
3          SB   {False: 19682, True: 318}
4          ST   {False: 19577, True: 423}
5          AF   {False: 19554, True: 446}
6  normal_ecg  {False: 12299, True: 7701}


Creating the folders for the classification task

In [9]:
import os
import shutil
labels_list = ["1dAVb","RBBB","LBBB","SB","ST","AF","normal_ecg", "not_normal_ecg"]

dest_root_fldr = r".\data"
src_path = r"..\data\code-15\exams_part0_imgs"
normal_count = 0
not_normal_count = 0
disease_count = 0

shutil.rmtree(dest_root_fldr, ignore_errors = True)
os.makedirs(dest_root_fldr, exist_ok = True)

for flr in labels_list:
    os.makedirs(dest_root_fldr+"\\"+f"{flr}")


for i,idx in enumerate(exam_id):
    try:
        if labels_arr[i,6]==1:
            shutil.copy(src_path+fr"\{idx}.jpg",os.path.join(dest_root_fldr, "normal_ecg"))
            normal_count += 1

        else:
            if np.count_nonzero(labels_arr[i])==0:
                shutil.copy(src_path+fr"\{idx}.jpg", os.path.join(dest_root_fldr, "not_normal_ecg"))
                not_normal_count +=1

            else:
                clas = labels_list[np.where(labels_arr[i]==1)[0][0]]
                shutil.copy(os.path.join(src_path,f'{idx}.jpg'), os.path.join(dest_root_fldr, f"{clas}"))
                disease_count += 1

    except Exception as e:
        print(e)

print("normal count", normal_count)
print("not normal count", not_normal_count)
print("disease count", disease_count)

normal count 7701
not normal count 10115
disease count 2184


In [46]:
np.count_nonzero(labels_arr)

10108

### Verifying the exam_id from the hdf5 and the csv file

In [152]:
pd = pd_results[pd_results["trace_file"]=="exams_part0.hdf5"]
exam_id_csv = pd.index.to_list()

In [149]:
exam_id_hdf5 = np.delete(np.array(f["exam_id"]),np.where(np.array(f["exam_id"])==0)).tolist()

In [153]:
exam_id_csv.sort()
exam_id_hdf5.sort()

In [147]:
exam_id_csv

In [158]:
import csv
with open("verify.csv", "w", newline='') as f:
    writer = csv.writer(f)

    for i, j in zip(exam_id_csv, exam_id_hdf5):
        writer.writerow([i, j])

print("written the csv file")    

written the csv file
